In [1]:
import pandas as pd
import pandas
import datetime as dt
from datetime import datetime
from urllib.error import HTTPError
#import requests
#from bs4 import BeautifulSoup
import re
import csv
import numpy as np
import warnings
from sklearn import datasets, linear_model, metrics
import pickle
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import statsmodels.api as sm
from scipy import stats

In [178]:
with open('exo_df_days.pickle', 'rb') as handle:
    exo_df_days = pickle.load(handle)

with open('cons_cyc_df_years.pickle', 'rb') as handle:
    cons_cyc_df_years = pickle.load(handle)
   


In [179]:
amzn = exo_df_days['AMZN']

amzn

,Close,Quarterly Revenue(Millions of US $),PE Ratio,Interest_rate
2006-01-03,47.58,9520.0,6.32,4.34
2006-01-04,47.25,9520.0,6.32,4.22
2006-01-05,47.65,9520.0,6.32,4.24
2006-01-06,47.87,9520.0,6.32,4.22
2006-01-09,47.08,9520.0,6.32,4.25
...,...,...,...,...
2017-11-06,1120.66,43744.0,12.20,1.16
2017-11-07,1123.17,43744.0,12.20,1.16
2017-11-08,1132.88,43744.0,12.20,1.16
2017-11-09,1129.13,43744.0,12.20,1.16


In [181]:
def ts_supervsd(nlags = 1):
    for i in amzn.columns:
        amzn.rename(columns = {i:i+'[t]'}, inplace = True) 
        for j in range(1,nlags+1):
            amzn[i.split('[t]')[0]+ '[t-' + str(j) + ']'] = amzn[i + '[t]'].shift(j)
    return amzn
    
amzn_lag = ts_supervsd(nlags = 1)    

In [ ]:
rename_dict = {}
for cols in amzn:
    rename_dict[cols] = cols + '[t]'
    #amzn.rename(columns = {cols:cols+'[t]'}, inplace = True)
amzn.rename(columns = rename_dict, inplace = True)


In [185]:
cols = ['Close[t]','Close[t-1]','Interest_rate[t]','Interest_rate[t-1]']

for col in cols:
    amzn[col + '_7_min'] = amzn[col].rolling(7).min()
    amzn[col + '_7_max'] = amzn[col].rolling(7).max()
    amzn[col + '_7_mean'] = amzn[col].rolling(7).mean()
    amzn[col + '_7_std'] = amzn[col].rolling(7).std()
    amzn[col + '_14_min'] = amzn[col].rolling(14).min()
    amzn[col + '_14_max'] = amzn[col].rolling(14).max()
    amzn[col + '_14_mean'] = amzn[col].rolling(14).mean()
    amzn[col + '_14_std'] = amzn[col].rolling(14).std()


In [187]:
amzn.head(20)

,Close[t],Quarterly Revenue(Millions of US $)[t],PE Ratio[t],Interest_rate[t],Close[t-1],Quarterly Revenue(Millions of US $)[t-1],PE Ratio[t-1],Interest_rate[t-1],Close[t]_7_min,Close[t]_7_max,...,Interest_rate[t]_14_mean,Interest_rate[t]_14_std,Interest_rate[t-1]_7_min,Interest_rate[t-1]_7_max,Interest_rate[t-1]_7_mean,Interest_rate[t-1]_7_std,Interest_rate[t-1]_14_min,Interest_rate[t-1]_14_max,Interest_rate[t-1]_14_mean,Interest_rate[t-1]_14_std
2006-01-03,47.58,9520.0,6.32,4.34,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2006-01-04,47.25,9520.0,6.32,4.22,47.58,9520.0,6.32,4.34,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2006-01-05,47.65,9520.0,6.32,4.24,47.25,9520.0,6.32,4.22,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2006-01-06,47.87,9520.0,6.32,4.22,47.65,9520.0,6.32,4.24,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2006-01-09,47.08,9520.0,6.32,4.25,47.87,9520.0,6.32,4.22,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2006-01-10,45.65,9520.0,6.32,4.24,47.08,9520.0,6.32,4.25,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2006-01-11,44.93,9520.0,6.32,4.24,45.65,9520.0,6.32,4.24,44.93,47.87,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2006-01-12,44.36,9520.0,6.32,4.28,44.93,9520.0,6.32,4.24,44.36,47.87,...,NaN,NaN,4.22,4.34,4.250000,0.041231,NaN,NaN,NaN,NaN
2006-01-13,44.40,9520.0,6.32,4.30,44.36,9520.0,6.32,4.28,44.36,47.87,...,NaN,NaN,4.22,4.28,4.241429,0.020354,NaN,NaN,NaN,NaN
2006-01-17,44.00,9520.0,6.32,4.32,44.40,9520.0,6.32,4.30,44.00,47.87,...,NaN,NaN,4.22,4.30,4.252857,0.027516,NaN,NaN,NaN,NaN
